## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Importing sys module to have a path for config module,
# so it doesn't have to be added to each folder everytime.
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,78,75,5.75,broken clouds
1,1,Lorengau,PG,-2.0226,147.2712,81.34,81,100,13.31,moderate rain
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,60,75,8.05,broken clouds
3,3,Meybod,IR,32.2501,54.0166,84.22,17,6,11.61,clear sky
4,4,Alofi,NU,-19.0595,-169.9187,78.69,94,75,3.44,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,78,75,5.75,broken clouds
1,1,Lorengau,PG,-2.0226,147.2712,81.34,81,100,13.31,moderate rain
3,3,Meybod,IR,32.2501,54.0166,84.22,17,6,11.61,clear sky
4,4,Alofi,NU,-19.0595,-169.9187,78.69,94,75,3.44,broken clouds
7,7,Casambalangan,PH,18.3765,122.1274,79.29,88,8,3.96,clear sky
9,9,Albany,US,42.6001,-73.9662,75.31,36,46,3.49,scattered clouds
10,10,Kapaa,US,22.0752,-159.3190,83.48,68,1,3.00,clear sky
11,11,Winneba,GH,5.3511,-0.6231,82.87,83,36,7.02,scattered clouds
12,12,Rikitea,PF,-23.1203,-134.9692,76.42,82,100,18.92,light rain
13,13,Vallenar,CL,-28.5708,-70.7581,78.75,15,81,5.08,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# No Nulls found.

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df_clean = preferred_cities_df.dropna(how="any")
preferred_cities_df_clean.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.36,78,75,5.75,broken clouds
1,1,Lorengau,PG,-2.0226,147.2712,81.34,81,100,13.31,moderate rain
3,3,Meybod,IR,32.2501,54.0166,84.22,17,6,11.61,clear sky
4,4,Alofi,NU,-19.0595,-169.9187,78.69,94,75,3.44,broken clouds
7,7,Casambalangan,PH,18.3765,122.1274,79.29,88,8,3.96,clear sky
9,9,Albany,US,42.6001,-73.9662,75.31,36,46,3.49,scattered clouds
10,10,Kapaa,US,22.0752,-159.3190,83.48,68,1,3.00,clear sky
11,11,Winneba,GH,5.3511,-0.6231,82.87,83,36,7.02,scattered clouds
12,12,Rikitea,PF,-23.1203,-134.9692,76.42,82,100,18.92,light rain
13,13,Vallenar,CL,-28.5708,-70.7581,78.75,15,81,5.08,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
1,Lorengau,PG,81.34,moderate rain,-2.0226,147.2712,
3,Meybod,IR,84.22,clear sky,32.2501,54.0166,
4,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,
7,Casambalangan,PH,79.29,clear sky,18.3765,122.1274,
9,Albany,US,75.31,scattered clouds,42.6001,-73.9662,
10,Kapaa,US,83.48,clear sky,22.0752,-159.3190,
11,Winneba,GH,82.87,scattered clouds,5.3511,-0.6231,
12,Rikitea,PF,76.42,light rain,-23.1203,-134.9692,
13,Vallenar,CL,78.75,broken clouds,-28.5708,-70.7581,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# Printing hotel_df.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Lorengau,PG,81.34,moderate rain,-2.0226,147.2712,Lorengau Harbourside Hotel
3,Meybod,IR,84.22,clear sky,32.2501,54.0166,Traditional Hotel Restaurant Haji Malek
4,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
7,Casambalangan,PH,79.29,clear sky,18.3765,122.1274,Villa Saturnina
9,Albany,US,75.31,scattered clouds,42.6001,-73.9662,
10,Kapaa,US,83.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Winneba,GH,82.87,scattered clouds,5.3511,-0.6231,Windy Lodge & Beach Resort
12,Rikitea,PF,76.42,light rain,-23.1203,-134.9692,Pension Maro'i
13,Vallenar,CL,78.75,broken clouds,-28.5708,-70.7581,Humacao Bed & Breakfast


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Lorengau,PG,81.34,moderate rain,-2.0226,147.2712,Lorengau Harbourside Hotel
3,Meybod,IR,84.22,clear sky,32.2501,54.0166,Traditional Hotel Restaurant Haji Malek
4,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
7,Casambalangan,PH,79.29,clear sky,18.3765,122.1274,Villa Saturnina
10,Kapaa,US,83.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Winneba,GH,82.87,scattered clouds,5.3511,-0.6231,Windy Lodge & Beach Resort
12,Rikitea,PF,76.42,light rain,-23.1203,-134.9692,Pension Maro'i
13,Vallenar,CL,78.75,broken clouds,-28.5708,-70.7581,Humacao Bed & Breakfast
17,Hays,US,85.35,overcast clouds,30.0502,-98.0336,Sinya on Lone Man Creek


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))